In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from htools import *

In [3]:
hdir('a')

{'capitalize': 'method',
 'casefold': 'method',
 'center': 'method',
 'count': 'method',
 'encode': 'method',
 'endswith': 'method',
 'expandtabs': 'method',
 'find': 'method',
 'format': 'method',
 'format_map': 'method',
 'index': 'method',
 'isalnum': 'method',
 'isalpha': 'method',
 'isascii': 'method',
 'isdecimal': 'method',
 'isdigit': 'method',
 'isidentifier': 'method',
 'islower': 'method',
 'isnumeric': 'method',
 'isprintable': 'method',
 'isspace': 'method',
 'istitle': 'method',
 'isupper': 'method',
 'join': 'method',
 'ljust': 'method',
 'lower': 'method',
 'lstrip': 'method',
 'maketrans': 'method',
 'partition': 'method',
 'replace': 'method',
 'rfind': 'method',
 'rindex': 'method',
 'rjust': 'method',
 'rpartition': 'method',
 'rsplit': 'method',
 'rstrip': 'method',
 'split': 'method',
 'splitlines': 'method',
 'startswith': 'method',
 'strip': 'method',
 'swapcase': 'method',
 'title': 'method',
 'translate': 'method',
 'upper': 'method',
 'zfill': 'method'}

In [4]:
%%lax
a

UsageError: Cell magic `%%lax` not found.


In [5]:
from htools import magics

In [6]:
%%mute
3

In [7]:
%%hush
3
4

4

In [8]:
import numpy as np
import pandas as pd

In [12]:
df = pd.DataFrame(np.arange(36).reshape(9, 4), columns=list('abcd'))
df

,a,b,c,d
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
6,24,25,26,27
7,28,29,30,31
8,32,33,34,35


In [10]:
df.ends()

AttributeError: 'DataFrame' object has no attribute 'ends'

In [11]:
from htools import pd_tools

In [14]:
df.ends(2)

,a,b,c,d
0,0,1,2,3
1,4,5,6,7
7,28,29,30,31
8,32,33,34,35


In [17]:
df['e'] = list('abcaaabab')
df

,a,b,c,d,e
0,0,1,2,3,a
1,4,5,6,7,b
2,8,9,10,11,c
3,12,13,14,15,a
4,16,17,18,19,a
5,20,21,22,23,a
6,24,25,26,27,b
7,28,29,30,31,a
8,32,33,34,35,b


In [39]:
v1 = df.groupby('e').a.value_counts()
v1

e  a 
a  0     1
   12    1
   16    1
   20    1
   28    1
b  4     1
   24    1
   32    1
c  8     1
Name: a, dtype: int64

In [40]:
v2 = df.groupby('e').a.value_counts(True)
v2

e  a 
a  0     0.200000
   12    0.200000
   16    0.200000
   20    0.200000
   28    0.200000
b  4     0.333333
   24    0.333333
   32    0.333333
c  8     1.000000
Name: a, dtype: float64

In [48]:
df.groupby('e').a.value_counts()

e  a 
a  0     1
   12    1
   16    1
   20    1
   28    1
b  4     1
   24    1
   32    1
c  8     1
Name: a, dtype: int64

In [43]:
pd.merge(v1, v2, left_index=True, right_index=True)

a_x       a_y
e a                
a 0     1  0.200000
  12    1  0.200000
  16    1  0.200000
  20    1  0.200000
  28    1  0.200000
b 4     1  0.333333
  24    1  0.333333
  32    1  0.333333
c 8     1  1.000000

In [53]:
from functools import wraps

In [165]:
import inspect
from types import FunctionType

In [237]:
def identity_fn(x):
    return x

In [686]:
select(hdir(Foo), ['c', 'f', 'i']), select(hdir(Foo), drop=['zzz'])

({'c': 'method', 'f': 'method', 'i': 'method'},
 {'c': 'method',
  'd': 'method',
  'f': 'method',
  'g': 'method',
  'h': 'method',
  'i': 'method'})

In [699]:
ms = ['a', '_b', 'c_', '__d', '__e__']
for m in ms:
    print(m, m.startswith('_'), m.startswith('__'))

a False False
_b True False
c_ False False
__d True True
__e__ True True


In [748]:
list(hdir(Foo))

['c', 'd', 'f', 'g', 'h', 'i']

In [813]:
dict.fromkeys(ms, True)

{'a': True, '_b': True, 'c_': True, '__d': True, '__e__': True}

In [858]:
def wrapmethods_dev(*decorators, methods=(), internals=False):
    """Class wrapper that applies 1 or more decorators to every non-magic
    method. For example, we often want @debug to be applied to many different
    methods. Magic methods and properties are excluded.

    Parameters
    ----------
    decorators: callable
        1 or more decorators to apply to methods within a class. By default,
        methods with 1 or 2 leading underscores are excluded.
    methods: Iterable[str]
        Names of methods to wrap if you don't want to wrap all of them. 
        Internal methods can be wrapped but magic methods and properties 
        cannot.
    internals: bool
        If True, apply decorators to methods named with leading single 
        underscores. This will be ignored if `methods` is specified.
    """
    def wrapper(cls):
        special_methods = (staticmethod, classmethod)
        to_wrap = dict.fromkeys(methods, True) if methods \
                  else hdir(cls, False, internals=internals)
        for attr, is_method in to_wrap.items():
            f = cls.__dict__[attr]
            mtype = type(f)
            if not is_method or isinstance(f, property):
                continue
            
            # Classmethod and staticmethod decorators need to be applied last.
            final_wrapper = identity_fn
            if isinstance(f, special_methods):
                final_wrapper = type(f)
                f = f.__func__
                
            for d in decorators:
                f = d(f)
            setattr(cls, attr, final_wrapper(f))
        return cls
    return wrapper

In [817]:
def dummy_deco(func):
    @wraps(func)
    def wrapped(*args, **kwargs):
        print('Decorated by dummy.')
        return func(*args, **kwargs)
    return wrapped

In [849]:
@wrapmethods_dev(debug, dummy_deco, timer, methods=['d', '_e'], internals=False)
@auto_repr
class Foo:
    def __init__(self, a, b=3):
        self.a = a
        self.b = b
    
    def c(self):
        return self.a*self.b
    
    def d(self, x):
        return x*2
    
    def _e(self, y):
        return y*3
    
    @staticmethod
    def f():
        return 'in staticmethod f'
    
    @staticmethod
    def g(a):
        return 'in staticmethod g', a
    
    @classmethod
    def h(cls, s):
        print('in classmethod h')
        return cls(s[:-1], int(s[-1]))
    
    @property
    def i(self):
        return str(self.a) + str(self.b)
    
    @i.setter
    def i(self, val):
        self.a = int(val[0])
        self.b = int(val[1:])

d <class 'function'>
_e <class 'function'>


In [850]:
f = Foo(3, 4)
f

Foo(a=3, b=4)

In [851]:
f.i

'34'

In [852]:
f.c()

12

In [853]:
f.d(7)

Decorated by dummy.

CALLING Foo.d(x=7)

[TIMER]: d executed in approximately 0.000 seconds.



14

In [854]:
f._e(5)

Decorated by dummy.

CALLING Foo._e(y=5)

[TIMER]: _e executed in approximately 0.001 seconds.



15

In [855]:
f.f()

'in staticmethod f'

In [856]:
f.g(3)

('in staticmethod g', 3)

In [857]:
Foo.h('3a2')

in classmethod h


Foo(a='3a', b=2)

In [631]:
@wrapmethods_dev(debug, dummy_deco, timer, internals=True)
@auto_repr
class Foo:
    def __init__(self, a, b=3):
        self.a = a
        self.b = b
    
    def c(self):
        return self.a*self.b
    
    def d(self, x):
        return x*2
    
    def _e(self, y):
        return y*3
    
    @staticmethod
    def f():
        return 'in staticmethod f'
    
    @staticmethod
    def g(a):
        return 'in staticmethod g', a
    
    @classmethod
    def h(cls, s):
        print('in classmethod h')
        return cls(s[:-1], int(s[-1]))

In [632]:
f = Foo(10)
f

Foo(a=10, b=3)

In [633]:
f.c()

Decorated by dummy.

CALLING Foo.c()

[TIMER]: c executed in approximately 0.000 seconds.



30

In [634]:
f.d(1)

Decorated by dummy.

CALLING Foo.d(x=1)

[TIMER]: d executed in approximately 0.001 seconds.



2

In [635]:
f._e(22)

Decorated by dummy.

CALLING Foo._e(y=22)

[TIMER]: _e executed in approximately 0.000 seconds.



66

In [636]:
f.f()

Decorated by dummy.

CALLING Foo.f()

[TIMER]: f executed in approximately 0.000 seconds.



'in staticmethod f'

In [637]:
f.g(44)

Decorated by dummy.

CALLING Foo.g(a=44)

[TIMER]: g executed in approximately 0.000 seconds.



('in staticmethod g', 44)

In [638]:
Foo.h('3be4')

Decorated by dummy.

CALLING Foo.h(s='3be4')
in classmethod h

[TIMER]: h executed in approximately 0.001 seconds.



Foo(a='3be', b=4)

In [566]:
@auto_repr
class Bar:
    def __init__(self, a, b=3):
        self.a = a
        self.b = b
    
    def c(self):
        return self.a*self.b
    
    def d(self, x):
        return x*2
    
    def _e(self, y):
        return y*3
    
    @staticmethod
    def f():
        return 'in staticmethod f'
    
    @staticmethod
    def g(a):
        return 'in staticmethod g', a
    
    @classmethod
    @dummy_deco
    @debug
    def h(cls, s):
        return cls(s[:-1], int(s[-1]))

In [567]:
b = Bar(11)
b

Bar(a=11, b=3)

In [568]:
Bar.h('ab1')

Decorated by dummy.

CALLING Bar.h(s='ab1')


Bar(a='ab', b=1)

In [569]:
Bar.h('ab1')

Decorated by dummy.

CALLING Bar.h(s='ab1')


Bar(a='ab', b=1)

In [570]:
dir(Bar.h)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__wrapped__']

In [571]:
Bar.h

<bound method Bar.h of <class '__main__.Bar'>>

In [575]:
Bar.h('ha3')

Decorated by dummy.

CALLING Bar.h(s='ha3')


Bar(a='ha', b=3)

In [576]:
bhsig = inspect.signature(Bar.h.__wrapped__)
print(bhsig)
bhsig.bind_partial(*(Bar, 'ha3'), **{})

(cls, s)


<BoundArguments (cls=<class '__main__.Bar'>, s='ha3')>

In [578]:
fhsig = inspect.signature(Foo.h.__wrapped__)
print(fhsig)
fhsig.bind_partial(*(Foo, 'ha3'), **{})

(cls, s)


<BoundArguments (cls=<class '__main__.Foo'>, s='ha3')>